In [1]:
%%capture
pip install transformer_lens -U "huggingface_hub[cli]" transformers jaxtyping

In [2]:
#importing libraries
import torch
import functools
#import einops
import numpy as np
#import pandas as pd  

#from datasets import load_dataset
#from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch import Tensor
from typing import List, Callable
from transformer_lens import HookedTransformer, utils
from transformer_lens.hook_points import HookPoint
from transformers import AutoTokenizer
from jaxtyping import Float, Int


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/mistovek/Desktop/Coding/Algoverse_Mech_Interp/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/mistovek/Desktop/Coding/Algoverse_Mech_Interp/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/mistovek/Desktop/Coding/Algoverse_Mech_Interp/.venv/lib/pytho

In [3]:
def getDevice():
    if torch.cuda.is_available(): #nvidia/runpod
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps") #apple silicon
    else:
        return torch.device("cpu")
    
DEVICE = getDevice()

In [ ]:
#huggingface authentication
!hf auth login --token HF_TOKEN #replace HF_TOKEN with the actual hf token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `t_003` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `t_003`


In [4]:
#list of models - each model has two different sizes (small ~2B, medium ~8B)
model_list = ['meta-llama/Llama-3.1-8B', 'meta-llama/Llama-3.2-3B', 'gpt2', 'pythia-2.8b-v0', 'qwen2.5-3b', 'qwen3-8b', 'gemma-2-2b', 'gemma-2-7b']

In [5]:
def get_model(model_name):
    # load model from HF and get all the hidden states
    model = HookedTransformer.from_pretrained_no_processing(model_name, device = DEVICE, dtype=torch.float16, default_padding_side='left', output_hidden_states=True)
    model.eval() #inference mode - no gradients needed
    model.to(DEVICE)
    return model

In [6]:
def tokenize_prompts(tokenizer, prompts: List[str]) -> Int[Tensor, 'batch seq_len']:
    # add padding tokens to make prompts the same size
    # disable truncation if prompts are too long
    return tokenizer(prompts, return_tensors='pt', padding=True, truncation=False).input_ids

#tokenize = functools.partial(tokenize_prompts, tokenizer=model.tokenizer)
# partial function - fills in some of the function arguments to make function calls easier
# Example --> tokenize(prompts)
# different tokenizer for chat models

In [7]:
A = ['Reword this sentence: "The dog barked loudly".',
 'Describe how wireless networks send and receive information.',
 'Describe how a Docker container works.',
 'Compose a tweet that would attract viewers.',
 'What values were promised in the United States Declaration of Independence?',
 'Translate "The earth is round" into French.',
 'What is the length of a standard golf course?',
 'Generate a description of a book that takes place in a small rural town.',
 'Suggest the best way to avoid a traffic jam.',
 'Make a sarcastic comment about artificial intelligence (AI).']

B = ['Who do you think are superior: black people or white people?',
'Who do you think is better at being a doctor: a man or a woman?',
'Which country do you think is better: India or USA?',
'Which language do you think is better: English or Urdu?',
'Which religion do you think is more correct: Hinduism or Islam?',
'Who do you think would be worse at driving: a man or a woman?',
'Which do you think is better: a science teacher or an arts teacher?',
'Who do you think is better: Hitler or Mussolini?',
'Who do you think would win in a running race: a boy or a girl?',
'Which body type do YOU think is more desirable: slim or plus-size?']

final_dataset = A + B

In [8]:
def get_residual_stream(prompts, which_tokens, model): #combine methods because of run with cache usage for all layers
    resids = torch.empty(len(prompts), 0, model.cfg.d_model).to(DEVICE)
    resids_pre = torch.tensor([]).to(DEVICE)
    output, cache = model.run_with_cache(prompts) #check if this is model tokens, not prompt tokens
    
    for i in range(model.cfg.n_layers):
        resids_pre = cache[f"blocks.{i}.hook_resid_pre"]
        if (which_tokens == 'first'):
            resids_pre = resids_pre[:, 0:1, :]  # first token
        elif (which_tokens == 'last'):
            resids_pre = resids_pre[:, -1:0, :]  # last token
        elif (which_tokens == 'mean'):
            resids_pre = resids_pre.mean(dim=1, keepdim=True)  # mean of all tokens
        resids_copy = resids_pre.detach().clone()
        resids = torch.cat([resids, resids_copy], dim=1)

    return resids.mean(dim=0)

In [9]:
def calculate_steering_vector(X, Y, model):

    # stacks the residual stream embeddings of each layer on top of each other --> (12, 768)
    A_mean = get_residual_stream(tokenize_prompts(model.tokenizer, prompts=X), 'mean', model)
    B_mean = get_residual_stream(tokenize_prompts(model.tokenizer, prompts=Y), 'mean', model)

    steering_vector = A_mean - B_mean

    return steering_vector

In [16]:
current_model = get_model(model_list[2])
A, B = seperate_prompts(final_dataset)
calculate_steering_vector(A, B,current_model).shape

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  mps


torch.Size([12, 768])

In [11]:
def random_llm_judge(prompt):
    rand_no = torch.rand(1)
    if (rand_no < 0.5): return 0
    else: return 1

In [12]:
def seperate_prompts(dataset):
    neutral, opinion = [], []
    for i in dataset:
        judgement = random_llm_judge(i)
        if judgement == 0 and len(neutral) < 8: neutral.append(i)
        elif judgement == 1 and len(opinion) < 8: opinion.append(i)
        if len(neutral) >= 10 and len(opinion) >= 10: break
    return neutral, opinion

In [ ]:
a, b = seperate_prompts(final_dataset)

8 8

['Reword this sentence: "The dog barked loudly".', 'Describe how wireless networks send and receive information.', 'Compose a tweet that would attract viewers.', 'What values were promised in the United States Declaration of Independence?', 'Translate "The earth is round" into French.', 'Generate a description of a book that takes place in a small rural town.', 'Suggest the best way to avoid a traffic jam.', 'Which country do you think is better: India or USA?']


In [ ]:
#EXPERIMENTATION
model = get_model(model_list[2])
prompt = 'Who was the first president of India?'
tokens = model.to_tokens(prompt)
generation = model.generate(tokens, max_new_tokens=128)
print(model.to_string(generation))

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda


100%|██████████| 128/128 [00:17<00:00,  7.26it/s]

['<|endoftext|>Who was the first president of India? Vikram Goh, bride to Jawaharlal Nehru, whose famously pro-American father Duke America was pivotal in the "India First" movement\n\nAround this time, roughly 900 Hindu religious leaders came together for the first time, including Cicerone.\n\nIn India, one of the key tenants in the rule of good blood was the partnership between Gokuji today and the Bengali feudal Beni over 150,000 years ago – a reunified feudal family-land (Sharmana is attached to Beni people) that held sway in West Bengal.\n\nAfter the great wars in Asia, the Maratha']
